In [1]:
# Refactored end-to-end pipeline for Risk-Drift + Adversarial Fragility
# Save as pipeline_refactor.py or run inside a Jupyter notebook cell.

import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from pathlib import Path

# --- CONFIG --- #
DATA_PATH = "Health_Risk_Dataset.csv"
FEATURES = ['Respiratory_Rate','Oxygen_Saturation','O2_Scale','Systolic_BP',
            'Heart_Rate','Temperature','Consciousness','On_Oxygen']
CONTINUOUS_VITALS = ['Respiratory_Rate','Oxygen_Saturation','Systolic_BP','Heart_Rate','Temperature']

SAFE_RANGES = {
    'Respiratory_Rate': (12, 20),
    'Oxygen_Saturation': (94, 100),
    'Systolic_BP': (100, 140),
    'Heart_Rate': (60, 100),
    'Temperature': (36.0, 37.5)
}

CONSCIOUSNESS_MAP = {'A':0,'C':1,'P':2,'V':3,'U':4}
RISK_MAP = {'Normal':0,'Low':1,'Medium':2,'High':3}
INV_RISK_MAP = {v:k for k,v in RISK_MAP.items()}

# --- Load --- #
if not Path(DATA_PATH).exists():
    raise FileNotFoundError(f"{DATA_PATH} not found. Place the CSV in working directory.")
df = pd.read_csv(DATA_PATH, sep=None, engine="python")
df.columns = [c.strip() for c in df.columns]

# quick schema check
expected = {'Patient_ID','Risk_Level'}
if not expected.issubset(set(df.columns)):
    raise ValueError(f"CSV missing required columns. Found: {df.columns.tolist()}")

# --- Preprocess --- #
df = df.reset_index(drop=True)
df['Consciousness'] = df['Consciousness'].map(CONSCIOUSNESS_MAP)
df['Risk_Num'] = df['Risk_Level'].map(RISK_MAP)
df['On_Oxygen'] = pd.to_numeric(df['On_Oxygen'], errors='coerce').fillna(0).astype(int)

# drop rows with missing features (small realistic datasets — alternative: impute)
missing_count = df[FEATURES].isna().sum().sum()
if missing_count > 0:
    print(f"Dropping {df[FEATURES].isna().any(axis=1).sum()} rows with missing feature values.")
    df = df.loc[~df[FEATURES].isna().any(axis=1)].reset_index(drop=True)

X = df[FEATURES].copy()
y = df['Risk_Level'].copy()
y_num = df['Risk_Num'].copy()

# --- Train model --- #
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
# We scale continuous vitals + keep categorical columns as-is for model input scaling only
scaled_cols = CONTINUOUS_VITALS + ['O2_Scale','Consciousness','On_Oxygen']
X_train_scaled = scaler.fit_transform(X_train[scaled_cols])
X_test_scaled = scaler.transform(X_test[scaled_cols])

# Train on raw values (forest can handle different scales) — but we keep scaler for clustering
model = RandomForestClassifier(n_estimators=400, random_state=42)
model.fit(X_train, y_train)
print("RandomForest trained. Test accuracy:", round(model.score(X_test, y_test), 4))

# --- SHAP (optional) --- #
try:
    import shap
    explainer = shap.TreeExplainer(model)
    # compute sample shap values for a subset (speed)
    sample = X_test.sample(min(200, len(X_test)), random_state=1)
    shap_values = explainer.shap_values(sample)
    SHAP_AVAILABLE = True
    print("SHAP available.")
except Exception as e:
    SHAP_AVAILABLE = False
    print("SHAP not available:", e)

# --- Clustering (GMM) & Risk Drift --- #
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

cluster_input = scaler.transform(X[scaled_cols])
best_k = 4; best_score = -1
for k in range(3,9):
    g = GaussianMixture(n_components=k, random_state=42).fit(cluster_input)
    labels = g.predict(cluster_input)
    try:
        s = silhouette_score(cluster_input, labels)
    except Exception:
        s = -1
    if s > best_score:
        best_score = s; best_k = k

gmm = GaussianMixture(n_components=best_k, random_state=42).fit(cluster_input)
df['Cluster'] = gmm.predict(cluster_input)
centroids = gmm.means_
print(f"GMM chosen clusters: {best_k} (silhouette {best_score:.3f})")

cluster_risk = df.groupby('Cluster')['Risk_Num'].mean().reindex(range(best_k)).fillna(0).values
dist_matrix = np.linalg.norm(centroids[:, None] - centroids[None, :], axis=2)
similarity = np.exp(-dist_matrix)
risk_drift_cluster = []
for a in range(best_k):
    higher = cluster_risk > cluster_risk[a]
    if higher.sum()==0:
        risk_drift_cluster.append(0.0)
    else:
        risk_drift_cluster.append(float(similarity[a,higher].sum() / similarity[a].sum()))
risk_drift_cluster = np.array(risk_drift_cluster)
df['Risk_Drift'] = df['Cluster'].map(lambda c: float(risk_drift_cluster[c]))
df['Risk_Drift_Score'] = (100*df['Risk_Drift']).round(1)

# --- Probabilistic deterioration (KDE-based) + Monte Carlo --- #
from sklearn.neighbors import KernelDensity

kde_models = {}
for vital in CONTINUOUS_VITALS:
    vals = df[vital].values.reshape(-1,1)
    low, high = SAFE_RANGES[vital]
    residuals = []
    for v in vals.flatten():
        if v < low:
            residuals.append((low - v))
        elif v > high:
            residuals.append((v - high))
        else:
            residuals.append(abs(np.random.normal(loc=0.2, scale=0.12)))
    residuals = np.array(residuals).reshape(-1,1)
    try:
        kde = KernelDensity(kernel='gaussian', bandwidth=0.3).fit(np.log1p(residuals))
        kde_models[vital] = ('kde', kde)
    except Exception:
        mu, sigma = float(residuals.mean()), float(residuals.std()) if residuals.std()>1e-6 else 0.5
        kde_models[vital] = ('norm', (mu, sigma))

def sample_deterioration_single(base_row):
    simulated = base_row.copy()
    VITAL_BOUNDS = {
        'Respiratory_Rate': (4, 60),
        'Oxygen_Saturation': (40, 100),
        'Systolic_BP': (50, 220),
        'Heart_Rate': (30, 220),
        'Temperature': (34.0, 42.0)
    }
    for vital in CONTINUOUS_VITALS:
        low, high = SAFE_RANGES[vital]
        orig = float(base_row[vital])
        typ = kde_models[vital][0]
        if typ == 'kde':
            kde = kde_models[vital][1]
            sampled = np.expm1(kde.sample(1)).flatten()[0]
            sampled = max(0.01, float(sampled) * np.random.uniform(0.6,1.4))
        else:
            mu, sigma = kde_models[vital][1]
            sampled = max(0.01, np.random.normal(mu, sigma))
        if vital == "Oxygen_Saturation":
            new_val = orig - sampled
        elif vital == "Respiratory_Rate":
            new_val = orig + sampled
        elif vital == "Heart_Rate":
            new_val = orig + sampled * np.random.uniform(0.9,1.3)
        elif vital == "Systolic_BP":
            if orig < low:
                new_val = orig - sampled
            elif orig > high:
                new_val = orig + sampled * np.random.uniform(0.4,1.2)
            else:
                new_val = orig - sampled * np.random.uniform(0.2,0.9)
        elif vital == "Temperature":
            new_val = orig + sampled * np.random.uniform(0.2,0.6)
        lo, hi = VITAL_BOUNDS[vital]
        simulated[vital] = float(np.clip(new_val, lo, hi))
    return simulated

def monte_carlo_future_prob(X_df, n_sims=200, required_label="High"):
    results = []
    high_idx = list(model.classes_).index(required_label)
    for idx, row in X_df.iterrows():
        base_row = row.copy()
        probs_high = []
        for s in range(n_sims):
            sim_row = sample_deterioration_single(base_row)
            prob_high = model.predict_proba(pd.DataFrame([sim_row[FEATURES]]))[0][high_idx]
            probs_high.append(prob_high)
        probs = np.array(probs_high)
        results.append({
            'Patient_idx': idx,
            'Patient_ID': df.loc[idx,'Patient_ID'],
            'current_high_prob': model.predict_proba(pd.DataFrame([base_row[FEATURES]]))[0][high_idx],
            'future_high_prob_mean': float(probs.mean()),
            'future_high_prob_p95': float(np.percentile(probs,95)),
            'future_high_prob_p05': float(np.percentile(probs,5)),
            'future_high_prob_std': float(probs.std())
        })
    return pd.DataFrame(results)

# run Monte Carlo on a sample (speed). Increase n_sims or sample size if time available.
print("Running Monte Carlo sample (this can take a while)...")
mc_sample = monte_carlo_future_prob(X.sample(min(200,len(X)), random_state=1), n_sims=200)
print("Monte Carlo sample done.")

# --- Adversarial minimal perturbation (Nelder-Mead) w/ fallback random search --- #
try:
    from scipy.optimize import minimize
    SCIPY_AVAILABLE = True
except Exception:
    SCIPY_AVAILABLE = False

VITAL_BOUNDS = {
    'Respiratory_Rate': (4, 60),
    'Oxygen_Saturation': (40, 100),
    'Systolic_BP': (50, 220),
    'Heart_Rate': (30, 220),
    'Temperature': (34.0, 42.0)
}

def get_row_features_for_model(row):
    return row[FEATURES]

def adversarial_minimal_change_for_row(idx, required_prob=0.5, target_label="High", n_restarts=8, maxiter=300):
    base_full = df.loc[idx].copy()
    base = base_full[FEATURES].copy()
    base_vals = np.array([base[v] for v in CONTINUOUS_VITALS], dtype=float)
    target_index = list(model.classes_).index(target_label)
    lower = np.array([VITAL_BOUNDS[v][0] for v in CONTINUOUS_VITALS])
    upper = np.array([VITAL_BOUNDS[v][1] for v in CONTINUOUS_VITALS])

    def build_input(x):
        row_copy = base.copy()
        for i,v in enumerate(CONTINUOUS_VITALS):
            row_copy[v] = float(x[i])
        return row_copy

    def obj(x):
        if np.any(x < lower) or np.any(x > upper):
            return 1e6 + np.linalg.norm(x - base_vals)
        prob = model.predict_proba(pd.DataFrame([build_input(x)]))[0][target_index]
        dist = np.linalg.norm(x - base_vals)
        penalty = 200.0 * max(0.0, required_prob - prob)
        return dist + penalty

    best = {'success': False, 'dist': np.inf, 'prob': None, 'x': None}
    if SCIPY_AVAILABLE:
        for r in range(n_restarts):
            x0 = np.clip(base_vals + np.random.normal(scale=0.5, size=len(base_vals)), lower, upper)
            res = minimize(obj, x0, method='Nelder-Mead', options={'maxiter':maxiter, 'xatol':1e-3, 'fatol':1e-3, 'disp': False})
            x_opt = np.clip(res.x, lower, upper)
            prob = model.predict_proba(pd.DataFrame([build_input(x_opt)]))[0][target_index]
            dist = float(np.linalg.norm(x_opt - base_vals))
            if prob >= required_prob and dist < best['dist']:
                best.update({'success': True, 'dist': dist, 'prob': float(prob), 'x': x_opt.copy()})
    else:
        # Random-search fallback
        for t in range(3000):
            x_try = np.clip(base_vals + np.random.normal(scale=1.0, size=len(base_vals)), lower, upper)
            prob = model.predict_proba(pd.DataFrame([build_input(x_try)]))[0][target_index]
            dist = float(np.linalg.norm(x_try - base_vals))
            if prob >= required_prob and dist < best['dist']:
                best.update({'success': True, 'dist': dist, 'prob': float(prob), 'x': x_try.copy()})

    # if we didn't find successful flip, provide best prob found:
    if not best['success']:
        best_prob = -1; best_x = None
        for t in range(800):
            x_try = np.clip(base_vals + np.random.normal(scale=1.2, size=len(base_vals)), lower, upper)
            prob = model.predict_proba(pd.DataFrame([build_input(x_try)]))[0][target_index]
            if prob > best_prob:
                best_prob = float(prob); best_x = x_try.copy()
        best.update({'success': False, 'dist': float(np.linalg.norm(best_x - base_vals)), 'prob': float(best_prob), 'x': best_x})

    deltas = {v: float(best['x'][i] - base_vals[i]) for i,v in enumerate(CONTINUOUS_VITALS)} if best['x'] is not None else None
    new_vals = {v: float(best['x'][i]) for i,v in enumerate(CONTINUOUS_VITALS)} if best['x'] is not None else None

    return {
        'Patient_idx': idx, 'Patient_ID': df.loc[idx,'Patient_ID'],
        'success': best['success'], 'distance': best['dist'],
        'prob_high': best['prob'], 'deltas': deltas, 'new_values': new_vals
    }

# Run adversarial search on a sample (speed)
sample_idxs = list(X.sample(min(60,len(X)), random_state=2).index)
adv_results = [adversarial_minimal_change_for_row(i, required_prob=0.5, n_restarts=8) for i in sample_idxs]
adv_df = pd.DataFrame(adv_results)
adv_df_sorted = adv_df.sort_values(by=['success','distance','prob_high'], ascending=[False, True, False])

# Combine Monte Carlo and adversarial results for the same sampled patients
combined = pd.merge(
    adv_df, 
    mc_sample, 
    on=['Patient_idx', 'Patient_ID'],  # Join on both index and Patient_ID
    how='left'
)
combined['Risk_Drift_Score'] = combined['Patient_idx'].map(lambda i: df.loc[i,'Risk_Drift_Score'])
combined['Current_Risk'] = combined['Patient_idx'].map(lambda i: df.loc[i,'Risk_Level'])

# Save outputs
combined.to_csv("combined_fragility_and_mc_summary.csv", index=False)
adv_df_sorted.to_csv("adversarial_summary_sample.csv", index=False)
mc_sample.to_csv("monte_carlo_summary_sample.csv", index=False)

print("Saved files: combined_fragility_and_mc_summary.csv, adversarial_summary_sample.csv, monte_carlo_summary_sample.csv")
print("Top fragile patients (adversarial flip found with small distance):")
print(adv_df_sorted[adv_df_sorted['success']].sort_values('distance').head(10))

# If in a Jupyter environment, display small samples
try:
    from IPython.display import display
    display(adv_df_sorted.head(20))
    display(mc_sample.head(20))
    display(combined.head(20))
except Exception:
    pass

# End of pipeline

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bdee0b07-dd93-46b8-8522-cce134092e71' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>